# Bedrock のログを有効化してみよう

Knowledge bases for Amazon Bedrock では次のログが利用可能です。

1. [Model invocation logging](https://docs.aws.amazon.com/bedrock/latest/userguide/model-invocation-logging.html)
2. [Knowledge bases logging](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-bases-logging.html)

今回はそれぞれのログ記録を有効化し、各ログに記録されるデータを見てみましょう。

## Model invocation logging

Model invocation logging は LLM 全般の実行履歴を記録します。次の形式の生成を行なった場合におけるログを記録します。

- Text
- Image
- Embedding

[Using the console](https://docs.aws.amazon.com/bedrock/latest/userguide/model-invocation-logging.html)

### 有効化

ログの有効化は Terraform でサポートされていないため、マネジメントコンソールより有効化作業を行います。Bedrock コンソールより `設定` からモデル呼び出しのログ記録を有効化します。

以下の情報を指定します。設定を保存します。

- ロググループ名：`/aws/bedrock/devio-2024-invoke`
- サービスロール：`devio-2024-bedrock-log-role`

![](../images/05/01.png)

#### コピペ用

In [ ]:
/aws/bedrock/devio-2024-invoke
devio-2024-bedrock-log-role

## Knowledge bases logging

Knowledge bases logging はナレッジベースへのデータの取り込みおよび、エンべディング処理を記録するためのログです。

ナレッジベースごとに設定可能で、取り込みジョブが失敗したことを検知したいケースなどで有効です。

[Knowledge bases logging](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-bases-logging.html)

### 有効化

こちらも現状 Terraform ではサポートされていないため、コンソールから記録を有効にします。

Bedrock コンソールよりナレッジベースを選択し、 `Log Deliveries` の Edit をクリックします。

![](../images/05/02.png)

ログ配信のセクションで対象のロググループを選択し、設定を保存します。

※閲覧から指定すると自動的に入力されます

- ロググループ名：`/aws/bedrock/devio-2024-kb`

#### コピペ用

In [ ]:
/aws/bedrock/devio-2024-kb


![](../images/05/03.png)

## 動作確認

それではログがどういった形で記録されるのか、動作確認をしてみましょう。

[/rag/datasource.tf](../terraform/rag/datasource.tf) の 71 から 81 行目をコメントを外し、新しいオブジェクトを S3 にアップロードします。


#### コピペ用

In [ ]:
cd ~/sagemaker-user/DevelopersIO-2024-bedrock/terraform/rag
terraform apply --auto-approve


ナレッジベースとデータソースの再同期を行います。Bedrock コンソールから`ナレッジベース`、 `devio-2024-kb` をクリックします。データソースから `devio-2024-datasource` を選択し、`同期`を行います。

![](../images/05/04.png)

### /aws/bedrock/devio-2024-invoke

CloudWatch コンソールよりロググループを選択します。

- `/aws/bedrock/devio-2024-invoke`

ログの内容を見てみましょう。同期処理はまず初めに `test` という値でエンべディングができるかどうかを判定しています。


```json
{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-07-17T07:28:28Z",
    "accountId": "XXXXXXXXXXXX",
    "identity": {
        "arn": "arn:aws:sts::XXXXXXXXXXXX:assumed-role/devio-2024-kb-role/BedrockKnowledgeBaseCPSession-DGPJXQCIT2"
    },
    "region": "us-west-2",
    "requestId": "823503c9-0c4f-48eb-bef1-69d3085fbdfc",
    "operation": "InvokeModel",
    "modelId": "arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "test"
        },
        "inputTokenCount": 1
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJson": {
            "embedding": [
                0.41015625,
                0.93359375
            ],
            "inputTextTokenCount": 1
        }
    }
}
```

その次に実際のチャンクに分割されたデータに対して、エンべディング処理にかけられていることがわかります。

```json
{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-07-17T08:04:04Z",
    "accountId": "XXXXXXXXXXXX",
    "identity": {
        "arn": "arn:aws:sts::XXXXXXXXXXXX:assumed-role/devio-2024-kb-role/SJ0PMX4K2W-EmbeddingTask-4377b5b5-f3c4-4e22-a85c-8ff9f8faea25"
    },
    "region": "us-west-2",
    "requestId": "0ce45641-3990-498b-9cc0-1fc91e77188f",
    "operation": "InvokeModel",
    "modelId": "arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "# オフィス利用ガイドライン  1. 一般的なルール     - オフィスの開閉時間を遵守してください（例：平日 8:00-20:00）    - 身分証明書やセキュリティカードを常に携帯し、必要に応じて提示してください    - オフィス内では適切な服装を心がけてください    - 喫煙は指定された場所でのみ行ってください  2. セキュリティ     - 部外者を無断でオフィスに入れないでください    - 機密情報の取り扱いには十分注意し、不要な書類は適切に廃棄してください    - パソコンから離れる際は必ずロックしてください    - セキュリティに関する懸念事項は速やかに報告してください  3. 共有スペース     - 会議室やミーティングスペースは予約システムを利用し、使用後は整理整頓してください    - キッチンや休憩スペースは清潔に保ち、使用後は片付けてください    - 騒音に配慮し、他の従業員の作業を妨げないようにしてください  4. 設備・備品の利用     - オフィス設備や備品は丁寧に扱い、故障や不具合は速やかに報告してください    - 個人的な目的での過度な設備利用（プリンター等）は控えてください    - エネルギー節約のため、不要な照明や機器の電源はこまめに切ってください  5. 健康・安全     - オフィス内での事故や怪我は直ちに報告してください    - 緊急時の避難経路と集合場所を把握しておいてください    - 体調不良時は無理せず、適切な対応（在宅勤務、休暇取得等）を取ってください  6. コミュニケーション     - 同僚への敬意を持ち、ハラスメントのない職場環境を維持してください    - オープンスペースでの長時間の私的な会話は控えてください    - 重要な連絡事項は適切な方法（メール、掲示板等）で共有してください  7. リモートワーク     - リモートワーク時も、通常のオフィスワーク同様のプロ意識を持って業務に取り組んでください    - オンライン会議の際は、適切な環境（背景、照明、音声）を整えてください  8. 環境への配慮     - ゴミの分別を徹底し、リサイクルに協力してください    - 紙の使用を最小限に抑え、可能な限りデジタル化を推進してください  9. 来客対応     - 来客の際は礼儀正しく対応し、必要に応じて適切な案内をしてください    - 機密情報が見える場所に来客を案内する際は十分注意してください  10. ガイドラインの遵守     - このガイドラインに違反する行為を見かけた場合は、適切な方法で報告してください     - ガイドラインは定期的に見直され、更新される可能性があります。最新版を常に確認してください  このガイドラインは、快適で生産的なオフィス環境を維持するためのものです。皆様のご協力をお願いいたします。"
        },
        "inputTokenCount": 579
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJson": {
            "embedding": [
                1.392622,
                0.415521,
            ],
            "inputTextTokenCount": 579
        }
    }
}
```

### /aws/bedrock/devio-2024-kb

CloudWatch コンソールよりロググループを選択します。

- `/aws/bedrock/devio-2024-kb`

ログの内容を見てみましょう。同期ジョブは `INGESTION_JOB_STARTED` からスタートします。

```json
{
    "event_timestamp": 1721203442741,
    "event": {
        "ingestion_job_id": "SJ0PMX4K2W",
        "data_source_id": "PLDVDQHSKK",
        "ingestion_job_status": "INGESTION_JOB_STARTED",
        "knowledge_base_arn": "arn:aws:bedrock:us-west-2:XXXXXXXXXXXX:knowledge-base/DGPJXQCIT2",
        "resource_statistics": {
            "number_of_resources_updated": 0,
            "number_of_resources_ingested": 0,
            "number_of_resources_scheduled_for_update": 0,
            "number_of_resources_scheduled_for_ingestion": 0,
            "number_of_resources_scheduled_for_metadata_update": 0,
            "number_of_resources_deleted": 0,
            "number_of_resources_with_metadata_updated": 0,
            "number_of_resources_failed": 0,
            "number_of_resources_scheduled_for_deletion": 0
        }
    },
    "event_version": "1.0",
    "event_type": "StartIngestionJob.StatusChanged",
    "level": "INFO"
}
```

`SCHEDULED_FOR_INGESTION`, `CRAWLING_COMPLETED` からデータのクローリングが行われます。

```json
{
    "event_timestamp": 1721203443084,
    "event": {
        "ingestion_job_id": "SJ0PMX4K2W",
        "document_location": {
            "type": "S3",
            "s3_location": {
                "uri": "s3://devio-2024-kb-datasource-XXXXXXXXXXXX-us-west-2/all/office-guideline.md"
            }
        },
        "data_source_id": "PLDVDQHSKK",
        "status_reasons": [],
        "knowledge_base_arn": "arn:aws:bedrock:us-west-2:XXXXXXXXXXXX:knowledge-base/DGPJXQCIT2",
        "status": "SCHEDULED_FOR_INGESTION"
    },
    "event_version": "1.0",
    "event_type": "StartIngestionJob.ResourceStatusChanged",
    "level": "INFO"
}
{
    "event_timestamp": 1721203443084,
    "event": {
        "ingestion_job_id": "SJ0PMX4K2W",
        "data_source_id": "PLDVDQHSKK",
        "ingestion_job_status": "CRAWLING_COMPLETED",
        "knowledge_base_arn": "arn:aws:bedrock:us-west-2:XXXXXXXXXXXX:knowledge-base/DGPJXQCIT2",
        "resource_statistics": {
            "number_of_resources_updated": 0,
            "number_of_resources_ingested": 0,
            "number_of_resources_scheduled_for_update": 0,
            "number_of_resources_scheduled_for_ingestion": 1,
            "number_of_resources_scheduled_for_metadata_update": 0,
            "number_of_resources_deleted": 0,
            "number_of_resources_with_metadata_updated": 0,
            "number_of_resources_failed": 0,
            "number_of_resources_scheduled_for_deletion": 0
        }
    },
    "event_version": "1.0",
    "event_type": "StartIngestionJob.StatusChanged",
    "level": "INFO"
}
```

クローリングが完了したのち、エンべディングが行われます。

```json
{
    "event_timestamp": 1721203444349,
    "event": {
        "ingestion_job_id": "SJ0PMX4K2W",
        "document_location": {
            "type": "S3",
            "s3_location": {
                "uri": "s3://devio-2024-kb-datasource-XXXXXXXXXXXX-us-west-2/all/office-guideline.md"
            }
        },
        "data_source_id": "PLDVDQHSKK",
        "status_reasons": [],
        "knowledge_base_arn": "arn:aws:bedrock:us-west-2:XXXXXXXXXXXX:knowledge-base/DGPJXQCIT2",
        "status": "EMBEDDING_STARTED"
    },
    "event_version": "1.0",
    "event_type": "StartIngestionJob.ResourceStatusChanged",
    "level": "INFO"
}
{
    "event_timestamp": 1721203444769,
    "event": {
        "ingestion_job_id": "SJ0PMX4K2W",
        "document_location": {
            "type": "S3",
            "s3_location": {
                "uri": "s3://devio-2024-kb-datasource-XXXXXXXXXXXX-us-west-2/all/office-guideline.md"
            }
        },
        "data_source_id": "PLDVDQHSKK",
        "status_reasons": [],
        "knowledge_base_arn": "arn:aws:bedrock:us-west-2:XXXXXXXXXXXX:knowledge-base/DGPJXQCIT2",
        "status": "EMBEDDING_COMPLETED"
    },
    "event_version": "1.0",
    "event_type": "StartIngestionJob.ResourceStatusChanged",
    "level": "INFO"
}
```

最後にエンべディングされたデータに対し、インデックス化が行われていることがわかります。

```json
{
    "event_timestamp": 1721203445022,
    "event": {
        "ingestion_job_id": "SJ0PMX4K2W",
        "document_location": {
            "type": "S3",
            "s3_location": {
                "uri": "s3://devio-2024-kb-datasource-XXXXXXXXXXXX-us-west-2/all/office-guideline.md"
            }
        },
        "data_source_id": "PLDVDQHSKK",
        "status_reasons": [],
        "knowledge_base_arn": "arn:aws:bedrock:us-west-2:XXXXXXXXXXXX:knowledge-base/DGPJXQCIT2",
        "status": "INDEXING_STARTED"
    },
    "event_version": "1.0",
    "event_type": "StartIngestionJob.ResourceStatusChanged",
    "level": "INFO"
}
{
    "event_timestamp": 1721203445460,
    "event": {
        "ingestion_job_id": "SJ0PMX4K2W",
        "document_location": {
            "type": "S3",
            "s3_location": {
                "uri": "s3://devio-2024-kb-datasource-XXXXXXXXXXXX-us-west-2/all/office-guideline.md"
            }
        },
        "data_source_id": "PLDVDQHSKK",
        "status_reasons": [],
        "knowledge_base_arn": "arn:aws:bedrock:us-west-2:XXXXXXXXXXXX:knowledge-base/DGPJXQCIT2",
        "status": "INDEXING_COMPLETED"
    },
    "event_version": "1.0",
    "event_type": "StartIngestionJob.ResourceStatusChanged",
    "level": "INFO"
}
{
    "event_timestamp": 1721203445961,
    "event": {
        "ingestion_job_id": "SJ0PMX4K2W",
        "document_location": {
            "type": "S3",
            "s3_location": {
                "uri": "s3://devio-2024-kb-datasource-XXXXXXXXXXXX-us-west-2/all/office-guideline.md"
            }
        },
        "chunk_statistics": {
            "ignored": 0,
            "metadata_updated": 0,
            "failed_to_update_metadata": 0,
            "deleted": 0,
            "failed_to_delete": 0,
            "created": 1,
            "failed_to_create": 0
        },
        "data_source_id": "PLDVDQHSKK",
        "knowledge_base_arn": "arn:aws:bedrock:us-west-2:XXXXXXXXXXXX:knowledge-base/DGPJXQCIT2",
        "status": "INDEXED"
    },
    "event_version": "1.0",
    "event_type": "StartIngestionJob.ResourceStatusChanged",
    "level": "INFO"
}
```

上記のログから、同期ジョブは `クローリング`, `エンべディング`, `インデクシング` のステップで行われていることがわかります。

## まとめ

このセクションでは次のことを学びました。

- Knowledge bases for Amazon Bedrock で利用できるログは現状 2 つある
  - モデルの実行ログ
  - 同期ジョブのログ
- 各ログを有効化することで同期ジョブではどのような処理が行われているのかがわかった
  - モデルをエンべディングする前に `test` という文字列をエンべディングしていた
  - `クローリング`, `エンべディング`, `インデクシング` のステップで行われていた

Next: [クエリを分割しログを眺めてみる](./06.ipynb)